In [ ]:
s# To Import files from computer
from google.colab import files 
train = files.upload()


Saving train.csv to train (1).csv


In [ ]:
!pip install distance

# Exploratory Data Analysis

In [ ]:
# Exploratory Data Analysis

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

In [ ]:
# Reading data 

df = pd.read_csv("test.csv")
print("Number of data points - ", df.shape[0])

Number of data points -  3263


In [ ]:
df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
df.info()            # gives the count of null values in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [ ]:
# Distribution of classes or data points among output classes

df.groupby("target")['id'].count().plot.bar()
print(df.groupby("target")['id'].count())

KeyError: ignored

In [ ]:
print('>> Tweeted Tweet is not representing a Disaster (target = 0):\n  {}%'.format(100 - round(df['target'].mean()*100,2)))
print('>> Tweeted Tweet is Representing a Disaster (target =1):\n  {}%'.format(round(df['target'].mean()*100,2)))

>> Tweeted Tweet is not representing a Disaster (target = 0):
  57.03%
>> Tweeted Tweet is Representing a Disaster (target =1):
  42.97%



Checking for Null Values



In [ ]:
# checking for null values

nan_rows = df[df.isnull().any(1)]
print(nan_rows)

         id keyword location                                               text
0         0     NaN      NaN                 Just happened a terrible car crash
1         2     NaN      NaN  Heard about #earthquake is different cities, s...
2         3     NaN      NaN  there is a forest fire at spot pond, geese are...
3         9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4        11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan
...     ...     ...      ...                                                ...
3258  10861     NaN      NaN  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259  10865     NaN      NaN  Storm in RI worse than last hurricane. My city...
3260  10868     NaN      NaN  Green Line derailment in Chicago http://t.co/U...
3261  10874     NaN      NaN  MEG issues Hazardous Weather Outlook (HWO) htt...
3262  10875     NaN      NaN  #CityofCalgary has activated its Municipal Eme...

[1105 rows x 4 columns]


In [ ]:
# Filling the null values 

df= df.fillna('')
nan_rows = df[df.isnull().any(1)]
print(nan_rows)


Empty DataFrame
Columns: [id, keyword, location, text]
Index: []


# Basic Feature Extraction (before cleaning) 
Constructing few features before data preprocessing

*  freq_keyword = frequency of keyword
*  freq_location = frequency of loacation
*  keyword_len = string (or each character) length of  keyword
*  location _len = string length of location
*  text_len = string length of each text
*  keyword_word = no of words in keyword 
*  text_word = number of words in text
*  word_common = number of common words in keyword and text 
*  word_share = (word_common)/(text_word)
*  freq_total = frequency of groupby of keyword and location 
*  






In [ ]:

df['freq_keyword'] = df.groupby('keyword')['keyword'].transform('count')
df['freq_location'] = df.groupby('location')['location'].transform('count')
df['keyword_len'] = df['keyword'].str.len()
df['location_len'] = df['location'].str.len()
df['text_len'] = df['text'].str.len()
df['keyword_word'] = df['keyword'].apply(lambda row: len(row.split(" ")))
df['text_word'] = df['text'].apply(lambda row: len(row.split(" ")))

def normalized_word_common(row):
    w1 = set(map(lambda word: word.lower().strip(), row['keyword'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['text'].split(" ")))
    return 1.0* len(w1 & w2)
df['word_common'] = df.apply(normalized_word_common,axis=1)

def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['keyword'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['text'].split(" ")))    
    return 1.0 * len(w1 & w2) / len(w2)

df['word_share'] = df.apply(normalized_word_share, axis=1)

df['freq_total'] = df.groupby(['keyword', 'location'])['keyword'].transform('size')







In [ ]:
df.head(60)

# Saving the data to csv file 

In [ ]:
df.to_csv('test_before_cleaning.csv')

# EDA: Advanced Feature Extraction

In [ ]:

df2 = pd.read_csv('test.csv')
df2 = df2.fillna('')


In [ ]:
df2.head()

,id,keyword,location,text
0,0,,,Just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s..."
2,3,,,"there is a forest fire at spot pond, geese are..."
3,9,,,Apocalypse lighting. #Spokane #wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan



# Preprcesing of text

*   Removing html tags
*   Removing Punctuations
*   Performing stemming
*   Removing Stopwords
*   Expanding Contractions etc.
*   







In [ ]:
! pip install fuzzywuzzy

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
# This package is used for finding longest common subsequence between two strings
# you can write your own dp code for this
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

from nltk.corpus import stopwords

STOP_WORDS = stopwords.words("english")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x
    


Definition:

Token: You get a token by splitting sentence a space(splits sentences into words these words are called tokens)

Stop_Word : stop words as per NLTK.

Word : A token that is not a stop_word


*  common_word_count = no of common words in keyword and text after removing stopwords
*   text_word_count - total no of words in text = q2_tokens
*   text_stopword - no of stopwords in text = q2_stops
*   text_nonstopword - no of words in text other than stopwords = q2_words
*  cwc = common_word_count/ text_nonstopword
*  csc = text_stopword/ text_word_count
*  csmc = text_stopword/ text_nonstopword
*  last_word_eq - check if last word of keyword is same as last word of text
*  first_word_eq - check if first word of keyword is same as first word of text
*  





In [ ]:
def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Tweet and Keyword
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Tweet and Keyword
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Tweet and Keyword
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Tweet and Keyword
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    if len(q2_words) ==0:
       token_features[0] =10
       token_features[2] =10
    else:
       token_features[0] = common_word_count / len(q2_words)
       token_features[2] = len(q2_stops) / len(q2_words)    
    
    if len(q2_tokens) ==0:
       token_features[1] =20
    else:
       token_features[1] = len(q2_stops) / len(q2_tokens)
    
    token_features[3] = common_word_count
    token_features[4] = len(q2_tokens)
    token_features[5] = len(q2_stops)    
    token_features[6] = len(q2_words)
    
    if len(q1_tokens) ==0:
    
       # Last word of both question is same or not
        token_features[7] = 0
    
      # First word of both question is same or not
        token_features[8] = 0
    else:
    # Last word of both question is same or not
        token_features[7] = int(q1_tokens[0] == q2_tokens[-1])
    
    # First word of both question is same or not
        token_features[8] = int(q1_tokens[0] == q2_tokens[0])
  
    return token_features

def extract_features(df2):
    # preprocessing each question
    df2["keyword"] = df["keyword"].fillna("").apply(preprocess)
    df2["text"] = df["text"].fillna("").apply(preprocess)

    print("token features...")

# extracting the feature from new df2 and assigining or appended the new extracted features to original df

# Merging Features with dataset
    
    token_features = df2.apply(lambda x: get_token_features(x["keyword"], x["text"]), axis=1)
    

   #Computing Fuzzy Features and Merging with Dataset
    
    # do read this blog: http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
    # https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
    # https://github.com/seatgeek/fuzzywuzzy
    print("fuzzy features..")

    df2["cwc"]        = list(map(lambda x: x[0], token_features))
    df2["csc"]        = list(map(lambda x: x[1], token_features))
    df2["csmc"]       = list(map(lambda x: x[2], token_features))
   
    df2["text_word_count"]      = list(map(lambda x: x[4], token_features))
    df2["text_stopword"]        = list(map(lambda x: x[5], token_features))
    df2["text_nonstopword"]     = list(map(lambda x: x[6], token_features))

    df2["last_word_eq"]  = list(map(lambda x: x[7], token_features))
    df2["first_word_eq"] = list(map(lambda x: x[8], token_features))
   

    df2["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["keyword"], x["text"]), axis=1)
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().

    df2["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["keyword"], x["text"]), axis=1)
    df2["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["keyword"], x["text"]), axis=1)
    df2["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["keyword"], x["text"]), axis=1)
   # df2["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["keyword"], x["text"]), axis=1)
    return df2




In [ ]:
df2 = extract_features(df2)
    


token features...
fuzzy features..


In [ ]:
df2.head(60)

,id,keyword,location,text,cwc,csc,csmc,text_word_count,text_stopword,text_nonstopword,last_word_eq,first_word_eq,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio
0,0,,,just happened a terrible car crash,0.000000,0.333333,0.500000,6,2,4,0,0,0,0,0,0
1,2,,,heard about earthquake is different cities s...,0.000000,0.222222,0.285714,9,2,7,0,0,0,0,0,0
2,3,,,there is a forest fire at spot pond geese are...,0.000000,0.500000,1.000000,20,10,10,0,0,0,0,0,0
3,9,,,apocalypse lighting spokane wildfir,0.000000,0.000000,0.000000,4,0,4,0,0,0,0,0,0
4,11,,,typhoon soudelor kills 28 in china and taiwan,0.000000,0.250000,0.333333,8,2,6,0,0,0,0,0,0
5,12,,,we are shaking it is an earthquak,0.000000,0.714286,2.500000,7,5,2,0,0,0,0,0,0
6,21,,,they d probably still show more life than arse...,0.000000,0.384615,0.714286,13,5,7,0,0,0,0,0,0
7,22,,,hey how are you,0.000000,0.750000,3.000000,4,3,1,0,0,0,0,0,0
8,27,,,what a nice hat,0.000000,0.500000,1.000000,4,2,2,0,0,0,0,0,0
9,29,,,fuck off,0.000000,0.500000,1.000000,2,1,1,0,0,0,0,0,0


In [ ]:
df2.to_csv('testnfeatures_after_preprocessing.csv')

# Featurizing text data with tfidf weighted word-vectors



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [ ]:
df3 =pd.read_csv('test.csv')

In [ ]:
df3.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
from scipy.stats import uniform, truncnorm, randint

print(uniform(0.01,0.199))

In [ ]:

df3['keyword'] = df['keyword'].apply(lambda x: str(x))
df3['text'] = df['text'].apply(lambda x: str(x))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
t = list(df3['text'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(t)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each text to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [ ]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['text'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df3['q1_feats_m'] = list(vecs1)


100%|██████████| 3263/3263 [00:45<00:00, 72.31it/s]


In [ ]:
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)

In [ ]:
# text tfidif weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,28.954128,-21.511105,-3.469303,15.600830,13.010379,1.697923,84.505383,29.611390,22.404018,36.224932,-18.297465,23.941772,20.555334,-49.391069,-34.038718,31.137385,0.243204,19.839255,-33.734419,5.170629,29.702602,9.584692,-3.113056,20.911629,-26.686046,33.753424,8.582570,-15.380763,69.878343,-108.374117,-10.613490,8.667327,35.787030,-56.445454,-29.961505,-45.343800,57.189260,-59.260858,-79.378880,7.253037,...,14.289963,3.207758,-2.347520,23.866818,8.979151,-63.608751,27.246721,-1.402141,80.771131,-32.837289,56.619552,-47.331277,-13.902187,17.536692,-18.847065,-13.631451,-13.972108,3.434943,30.894864,-12.039361,78.934539,-10.224104,-21.200187,-5.073649,-11.546250,-64.253518,7.993286,30.405345,-14.838016,-39.253505,-18.268648,46.667951,14.385823,16.979114,16.825456,4.087669,8.004843,18.826909,23.763706,40.628766
1,25.269621,9.539629,-127.261414,-42.897729,123.614088,6.808554,104.142828,123.541424,78.563778,125.415496,-19.922479,-3.141657,76.232928,-50.769698,-1.411426,24.735637,-71.613426,17.202949,-65.628136,-30.843783,46.675943,-4.244756,-48.482558,33.433580,-30.246726,4.631195,-50.953841,-33.511216,77.431704,-75.740762,-3.984733,40.009471,69.030146,-60.710115,9.587962,-75.704771,98.500640,-87.684851,-160.780852,40.407624,...,84.222866,22.826723,-15.576600,36.014724,14.409822,-38.102800,49.620100,1.317689,61.711831,-76.219178,61.478048,-133.124709,6.233181,96.953805,-62.853296,-42.593580,25.965430,10.401568,13.343224,-72.129746,148.423120,-23.741878,-71.540982,-4.938113,-2.500674,-55.453331,1.321568,-34.407689,26.755620,-157.535120,-43.598078,-0.535778,34.696688,98.591888,-6.980470,-45.372115,16.488738,-13.638818,60.843353,80.633611
2,102.319203,-6.412488,-52.777348,-27.057129,93.060506,80.920439,84.583523,-42.629564,99.343330,121.118168,34.123374,1.658605,-10.864337,-98.943115,-68.647551,-20.065341,-60.333160,-79.111142,-51.335129,65.120818,14.916569,28.484546,-21.628196,-2.232250,-185.233552,118.537575,-10.596534,37.814353,64.328356,-181.494493,104.103037,-11.254745,-43.036332,-169.367990,-22.351397,-71.336544,112.098955,-71.141051,-192.446475,-56.254812,...,33.247451,113.779617,-6.371152,-15.111072,158.647712,-74.913216,142.200967,66.859366,83.616090,-44.587797,82.570099,-136.622206,-12.109850,100.986059,-59.696543,-33.504881,63.801579,-16.448219,47.195833,28.847658,133.741240,-171.259521,-95.353403,-34.371520,-12.131428,-120.524147,-35.893528,41.309197,25.609815,-25.843817,66.396542,-15.568521,-48.121708,35.768335,-60.049175,-21.940854,-19.440872,141.166020,69.317888,81.015415
3,-10.982039,-25.922097,2.823105,30.413317,-5.268434,5.062134,22.958576,32.652419,-10.708592,46.142717,37.731498,46.426503,62.244478,-31.119417,-31.363042,-5.513807,-24.148593,27.808517,-0.175966,-21.774320,15.220118,5.507034,14.732738,-2.477294,-16.517843,21.424973,-32.646934,-4.830914,-6.702926,-29.823359,11.379392,26.369427,37.403607,-49.037295,-19.842247,-67.551642,45.431527,-20.935335,-119.930006,22.833868,...,44.031492,57.703329,14.734437,-26.347546,33.345814,-11.210532,67.927213,6.445773,68.325803,1.493007,37.584659,-88.992137,-16.061937,18.912601,-0.408977,-90.737800,37.590670,48.946409,-22.253661,-38.757223,35.093636,-54.872455,10.772605,-59.037587,18.707490,4.235479,-43.455570,-28.487795,16.758103,19.106277,-21.919748,28.453778,9.288380,25.715577,47.149427,-11.653367,-5.040197,-3.054823,24.208777,36.665053
4,-10.970750,-40.557869,-21.116915,84.568180,32.170707,167.107039,18.010069,-28.548837,71.553353,71.992939,61.503954,-8.585110,15.264132,-41.053900,-77.925164,-65.811001,-3.830038,89.875995,-14.118770,1.935063,35.284226,-95.445607,23.453920,-65.411708,-66.906801,-17.640450,18.794535,-40.040994,-2.541546,-99.090639,39.096847,-73.873440,24.511690,-35.107854,42.935180,-74.482960,86.645453,-77.742183,-5.857198,

In [ ]:
df3_q1.to_csv('test_tdif_feature.csv')